# Try to read and convert the networks

For the Sioux network

In [1]:
import csv
import pandas as pd
import numpy as np

# reading a TNTP text file
def read_file_net(network_name, suffix="net"):
    assert suffix in ["net","node","trips", "flow"]
    with open("data/" + network_name + "/" + network_name + "_" + suffix + ".tntp") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter="\t")
        index_to_legend = {}
        data = {}
        metadata = {}
        line_count = 0
        for row in csv_reader:
            if len(row)==0:
                # The row is empty 
                continue 
            if "<" in row[0]:
                metadata_aux = row[0].split('<')[1].split('> ')
                if metadata_aux[0]=="END OF METADATA>":
                    continue
                metadata[metadata_aux[0]] = metadata_aux[1]
            else:
                if (suffix=="net" or suffix=="node" or suffix=="flow") and row[-1] == ";":
                    if line_count == 0:
                        # the first line is the legend
                        i = 0
                        for legend in row[:-1]:
                            # print(legend)
                            index_to_legend[i] = legend
                            data[legend] = []
                            i += 1
                    else:
                        i = 0
                        for point in row[:-1]:
                            data[index_to_legend[i]].append(point)
                            i += 1
                        # print(row)
                    line_count += 1
                elif(suffix=="trips"):
                    if row[0] == 'Origin ':
                        origin = int(row[1])
                        data[origin] = {}
                    else:
                        # print(row)
                        dest_array = row[0].split(";")
                        # print(dest_array)
                        for dest in dest_array[:-1]:
                            dest_tmp = dest.split(":")
                            data[origin][int(dest_tmp[0])] = float(dest_tmp[1])
                else:
                    print("This setence should never appear. Error")
    d = pd.DataFrame(data)
    return d, metadata

In [4]:
# net and node are ok
metadata = {}
network_name = "SiouxFalls"

for suffix in ["net","node","trips", "flow"]:
    print("---------" + suffix + "---------")
    d, metadata_aux = read_file_net(network_name, suffix)
    metadata.update(metadata_aux)
    print(d.head())
    print()
print(metadata)

---------net---------
  ~  Init node  Term node     Capacity  Length  Free Flow Time      B Power  \
0             1          2  25900.20064       6               6  0.15     4   
1             1          3  23403.47319       4               4  0.15     4   
2             2          1  25900.20064       6               6  0.15     4   
3             2          6  4958.180928       5               5  0.15     4   
4             3          1  23403.47319       4               4  0.15     4   

  Speed limit  Toll  Type  
0            0     0    1  
1            0     0    1  
2            0     0    1  
3            0     0    1  
4            0     0    1  

---------node---------
  Node       X       Y
0    1   50000  510000
1    2  320000  510000
2    3   50000  440000
3    4  130000  440000
4    5  220000  440000

---------trips---------
      1      2      3      4      5      6      7      8      9       10  \
1    0.0  100.0  100.0  500.0  200.0  300.0  500.0  800.0  500.0  1300.0